# Use a 2D convolutional neural net to identify the artist behind a painting

The title kind of says it all. This notebook is a 2D convolutional neural net (from scratch) to identify the artist behind a painting. There are over 50 artists with over 8000 paintings, some of them kind of similar in the technique, so this makes the training and identification sort of tricky. 

Having said that, I personally am a big fan of Ernst Ludwig Kirchner, so I was a little bit disappointed not to find him in the list of 'the best artworks of all time' (as you will see later). Here, just for completeness a nice painting of Kirchner (title: Nollendorfplatz): 

![](https://upload.wikimedia.org/wikipedia/commons/f/ff/Ernst_Ludwig_Kirchner_-_Nollendorfplatz.jpg)

In any case enjoy the notebook. Any ideas, suggestions or comments are of course more than welcome and appreciated. Have fun!

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import gridspec
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
# define a couple of constants
IMG_HEIGHT  = 300
IMG_WIDTH   = 300
BATCH_SIZE  = 32
EPOCHS      = 20
SEED        = 1234
strPath     = "../input/best-artworks-of-all-time/images/images/"

# Prepare data

Next part is to read the data for further usage. For this, predefined functions of Keras are used. I use 20% for validation. The image sizes are rather large since it increases accuracy later.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  strPath,
  validation_split = 0.2,
  subset     = "training",
  seed       = SEED,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  strPath,
  validation_split = 0.2,
  subset     = "validation",
  seed       = SEED,
  image_size = (IMG_HEIGHT, IMG_WIDTH),
  batch_size = BATCH_SIZE)


Get ( and print) the name of classes, i.e. the name of the artists.

In [ ]:
# get the artists names
class_names = train_ds.class_names

# define for later
num_classes = len(class_names)

print(class_names)

# Visualise some of the paintings

In [ ]:
plt.figure(figsize=(14, 14))
for images, labels in train_ds.take(1):
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

# Data augmentation

In [ ]:


data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.25),
    layers.experimental.preprocessing.RandomZoom(0.2), 
    layers.experimental.preprocessing.RandomTranslation(0.3,0.2), 
    layers.experimental.preprocessing.RandomContrast(0.2)
  ]
)

# Visualise data augmentation

Just make some plots of the results of the daa augmentation based on a random painting.

In [ ]:
plt.figure(figsize=(14, 14))
for images, _ in train_ds.take(1):
  for i in range(16):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

# Model

The model is a 2D convolutional neural network from scratch. Here one could probably still spend a little bit of time to improve the final results especially acuracy. One could think of using a more complex architecture or something predefined (e.g. ResNet, GoogLeNet, etc) and then attaching something in the end. 

In [ ]:

CONV = 3

model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, CONV, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, CONV, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, CONV, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])



As optimiser, I tried a couple of them, but RMSprop seemed to make the best job in terms of accuracy in the end. The choice for loss and metric seems kind of natural for this type of problem.

In [ ]:
#define model ingredients 
opti = tf.keras.optimizers.RMSprop(momentum=0.1) 
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) 
metr = ['accuracy']

#compile model 
model.compile(optimizer = opti, 
              loss = loss, 
              metrics = metr)

In [ ]:

history = model.fit(
  train_ds,
  validation_data = val_ds,
  epochs          = EPOCHS
)

In [ ]:
model.summary()


# Check loss and accuracy

To check for the accuracy but also of course to check on overfitting, the relevant quantities are plotted vs the number epochs.

In [ ]:
# get the values
acc      = history.history['accuracy']
val_acc  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']


# show the results
epochs_range = range(EPOCHS)


plt.rcParams.update({'font.size': 15})

fig = plt.figure(figsize=(12, 14))
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1]) 

ax0 = plt.subplot(gs[0])
plt.plot(epochs_range, acc, label='Training')
plt.plot(epochs_range, val_acc, label='Validation')
plt.legend(loc='lower right')
plt.title('Accuracy and loss')
plt.ylabel("Accuracy")
plt.grid(True)

ax1 = plt.subplot(gs[1], sharex = ax0)
plt.plot(epochs_range, loss, label='Training')
plt.plot(epochs_range, val_loss, label='Validation')
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.grid(True)

plt.subplots_adjust(hspace=.0)
plt.show()

# Conclusion

This is a simple version of a convolutional Neural Net. Results are not that awesome, but somewhat solid considering created from scratch. Overfitting seems to be on the smaller level, so this should be okay. Using pretrained models like ResNet50 could improve the performance, will probably try this later at some point. Any further ideas, suggestions on how to increase the accuracy are of course more than welcome.